In [1]:
from bubls.utils.rag_design.rag_building_blocks import RAGBuildingBlocks
from bubls.utils.evaluation.evaluate_llm import evaluate_llm_performance
from bubls.utils.evaluation.evaluate_embeddings import evaluate_retriever
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
import nest_asyncio
nest_asyncio.apply()
import logging
import sys
from metadata import lyft_10k, wiki_public_companies, papers_ridesharing

<jemalloc>: Unsupported system page size


In [2]:
Settings.llm = OpenAI(temperature=0.2, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(name="text-embedding-ada-002")

In [3]:
logging.basicConfig(
    stream=sys.stdout,
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

In [6]:
components_cfg = {
    "lyft_10k": lyft_10k,
    "wiki_public_companies": wiki_public_companies,
    "papers_ridesharing": papers_ridesharing,
}

In [7]:
rag = RAGBuildingBlocks(components_cfg)
rag.execute()

Loading data artifacts for lyft_10k
Loading LLM eval data for lyft_10k
Loading Nodes for lyft_10k
Loading nodes lyft_10k
Loading QA pairs for lyft_10k
Loading engines for lyft_10k
Loading Index for lyft_10k
Generating Query Engine for lyft_10k
Generating Retriever for lyft_10k
Generating data artifacts for wiki_public_companies
Loading wikipedia data wiki_public_companies
Generating LLM Eval Data for wiki_public_companies


/opt/conda/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:212: DeprecationWarning: Call to deprecated class DatasetGenerator. (Deprecated in favor of `RagDatasetGenerator` which should be used instead.)
  return cls(
/opt/conda/lib/python3.11/site-packages/llama_index/core/evaluation/dataset_generation.py:309: DeprecationWarning: Call to deprecated class QueryResponseDataset. (Deprecated in favor of `LabelledRagDataset` which should be used instead.)
  return QueryResponseDataset(queries=queries, responses=responses_dict)


Generating Nodes for wiki_public_companies
Generating QA Pairs for wiki_public_companies


100%|██████████| 37/37 [00:55<00:00,  1.51s/it]


Generating engines for wiki_public_companies
Generating Index for wiki_public_companies
Generating Query Engine for wiki_public_companies
Generating Retriever for wiki_public_companies
Loading data artifacts for papers_ridesharing
Loading LLM eval data for papers_ridesharing
Loading Nodes for papers_ridesharing
Loading nodes papers_ridesharing
Loading QA pairs for papers_ridesharing
Loading engines for papers_ridesharing
Loading Index for papers_ridesharing
Generating Query Engine for papers_ridesharing
Generating Retriever for papers_ridesharing


In [8]:
evaluate_llm_performance(rag.llm_eval_data["papers_ridesharing"], rag.query_engine["papers_ridesharing"])

  0%|          | 0/10 [00:00<?, ?it/s]

{'response_time': 3.1531354427337646, 'faithfulness': 1.0, 'relevancy': 1.0}

In [12]:
a=evaluate_retriever(rag.qa_pairs["papers_ridesharing"], rag.retriever["papers_ridesharing"])
print(a.is_hit.mean(), a.mrr.mean())

  0%|          | 0/17 [00:00<?, ?it/s]

0.9411764705882353 0.7450980392156863
